In [3]:
import keras
import tensorflow as tf
import pandas as pd
import csv
import statistics as st

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import contractions
contractions.fix("This can't be the worst, i'm sure")

C:\Users\nickr\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


'This cannot be the worst, i am sure'

In [4]:
df = pd.read_csv('Full Dataset.csv', encoding='utf8')
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5332 entries, 0 to 5369
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Company            5332 non-null   object
 1   Job Title          5332 non-null   object
 2   Employement Level  5332 non-null   int64 
 3   Review Title       5332 non-null   object
 4   Pros               5332 non-null   object
 5   Cons               5332 non-null   object
 6   Rating             5332 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 333.2+ KB


In [5]:
df.head()

,Company,Job Title,Employement Level,Review Title,Pros,Cons,Rating
0,eBay,Consultant,0,Good Company,The people and the company culture is Warm. Lo...,The company is very old fashioned and the proj...,5
1,eBay,Authentication Center Generalist,0,A step in the wrong direction,"Since eBay's acquisition of my employer, TCGpl...","Regrettably, eBay has made a mess of their acq...",2
2,eBay,Software Development Engineer (SDE),0,Good,Good company to work with,Cafeteria need to have more choice,5
3,eBay,Strategic Account Manager,1,Good Environment,"Great culture, good benefits, good options for...","Slow, risk adverse, reinvention in progress",4
4,eBay,Manager,1,Manager,Good benefits with good environment,Nothing really much to say,4


In [6]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')


print(df['Employement Level'].value_counts()[1])
#print(df['Employement Level'].value_counts()['1'])
unique_counts = df['Job Title'].value_counts()

print(len(unique_counts))

1079
1795


In [7]:
def getLongestContent(df, column):
    longest = 0
    longestContent = ""
    entry = ""
    for index, row in df.iterrows():
        pro = row[column].split()
        if(len(pro) > longest):
            longest = len(pro)
            longestContent = pro
            entry = row
    return entry#longest#, entry

In [8]:
sentenceHeaders = ['Review Title', 'Pros', 'Cons']
for header in sentenceHeaders:
    print(header, ": ", getLongestContent(df, header))

Review Title :  Company                                                         Amazon
Job Title                                              Receiver/Picker
Employement Level                                                    0
Review Title         I love it here it's a lot to learn in Amazon T...
Pros                 They help you learn a lot they teach you every...
Cons                 I really haven't seen any yet well I just real...
Rating                                                               5
Name: 1030, dtype: object
Pros :  Company                                                          Tesla
Job Title                                                Test Engineer
Employement Level                                                    0
Review Title                                             Test Engineer
Pros                 Working at Tesla can offer a unique and exciti...
Cons                 While there are many positive aspects of worki...
Rating                     

In [24]:
def preprocess_text(text):

    # Tokenize the text

    tokens = word_tokenize(contractions.fix(text.lower()))
    filtered_tokens = [token for token in tokens if token not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

def cleanUp(sentence):
    return contractions.fix(sentence).lower()

def sentimentScore(sentence, sent):
    sia = SentimentIntensityAnalyzer()
    return int(sia.polarity_scores(sentence)[sent] * 100)
    
cols = ['Review Title', 'Pros', 'Cons']
sents = ["neg", "neu", "pos", "compound"]

for col in cols:
    colToken = col + "token"
    for sent in sents:
        colsent = col + "-" + sent
        df[colsent] =  df[col].map(lambda s: sentimentScore(s, sent))
        print(colsent, " Done")
    df[col] = df[col].map(lambda s: cleanUp(s))
    df[colToken] = df[col].map(preprocess_text)
    print(col, " done.")
    

Review Title-neg  Done
Review Title-neu  Done
Review Title-pos  Done
Review Title-compound  Done
Review Title  done.
Pros-neg  Done
Pros-neu  Done
Pros-pos  Done
Pros-compound  Done
Pros  done.
Cons-neg  Done
Cons-neu  Done
Cons-pos  Done
Cons-compound  Done
Cons  done.


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5332 entries, 0 to 5369
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Company                5332 non-null   object
 1   Job Title              5332 non-null   object
 2   Employement Level      5332 non-null   int64 
 3   Review Title           5332 non-null   object
 4   Pros                   5332 non-null   object
 5   Cons                   5332 non-null   object
 6   Rating                 5332 non-null   int64 
 7   consTokens             5332 non-null   object
 8   Review Title-neg       5332 non-null   int64 
 9   Review Title-neu       5332 non-null   int64 
 10  Review Title-pos       5332 non-null   int64 
 11  Review Title-compound  5332 non-null   int64 
 12  Review Titletoken      5332 non-null   object
 13  Pros-neg               5332 non-null   int64 
 14  Pros-neu               5332 non-null   int64 
 15  Pros-pos               533

In [26]:
df.loc[368] #['Review Title']

Company                                                               eBay
Job Title                                                      Ex-employee
Employement Level                                                        0
Review Title                        you will enjoy training and that is it
Pros                     yes, you do not speak about any problems you a...
Cons                     the cons are being trained one way and getting...
Rating                                                                   1
consTokens               con trained one way getting training half told...
Review Title-neg                                                         0
Review Title-neu                                                        68
Review Title-pos                                                        31
Review Title-compound                                                   49
Review Titletoken                                           enjoy training
Pros-neg                 

In [27]:
df.to_csv('Dataset Reviews.csv', index=False)  

In [20]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
print(sia.polarity_scores("I hate this so much it's EVIL!"))

{'neg': 0.701, 'neu': 0.299, 'pos': 0.0, 'compound': -0.8857}
